혼잡한 관광지 발생시, 혼잡하지 않은 곳을 방문하기 원하는 수요를 위해 유사한 관광지 10곳을 추천하고자 함

한국관광공사에서 운영하는 '대한민국 구석구석'에서(https://korean.visitkorea.or.kr/search/search_list.do?keyword=%EC%A0%9C%EC%A3%BC%EB%8F%84) 제공하고 있는 관광지별 관련 카테고리 데이터를 수집한 다음 이를 이용해 문서유사도 측정을 통해 10곳을 추천하고자 함

[데이터셋 변수 설명]
- mecab : Mecab 라이브러리를 활용하기 위한 객체 생성
- file_data : 작성한 사용자 사전 파일을 읽어 Colab 환경에 설치된 Mecab 라이브러리에 추가하기 위한 변수, 한 줄 씩 읽옴
- poi : 관광지별 관련 카테고리를 저장한 데이터 셋
- stop_words : 의미 분석에 방해가 되는 불용어를 지정한 문자열
- vectorizer : Tf-idf 방식으로 벡터화
- cosine_sim : 코사인 유사도를 sklearn 라이브러리를 통해 구하기 위해 cosine_similarity 객체를 생성하기 위한 변수
- indices : 관광지명 순서별로 인덱스를 지정하기위해 Series 형태로 저장하기 위한 변수
- get_recommendation : 관광지명을 입력하면 유사한 10가지 관광지를 추천해주기 위한 함수

[전처리 과정 설명] 
- 대한민국 구석구석에서 제공하는 관광지별 카테고리를 Jeju_POI_withexplanation.xslx 로 저장함
- 해당 데이터의 전처리를 위해 Mecab의 기존 형태소 분석 결과를 확인하고 인생 샷(-> 인생샷)과 같이 의미를 이루기 위해서 묶여야할 음절들을 하나의 단어로 묶기 위한 사용자 사전을 작성함
- 불용어를 추가하고 사용자 사전을 사용했음에도 묶이지 않은 단어를 확인하여 추가적인 전처리 과정을 마침

[추천 시스템 구축 프로세스]
1. 관광지별 카테고리 데이터 수집
2. 전처리를 위한 사용자 사전 구축
3. Mecab 라이브러리를 활용한 자연어 처리
4. TF-IDF Vectorizor를 활용해, 문서유사도 기반 유사 관광지 추천

[기타 데이터 관련 설명]
- 대한민국 구석구석에서 제공하고 있는 카테고리 데이터는 한국관광공사에서 연관 단어 등을 수집하여 카테고리화한 것임

In [ ]:
# Mecab-ko 설치를 위한 깃 클론
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

In [ ]:
cd Mecab-ko-for-Google-Colab/

In [ ]:
# JPype1, konlpy 설치
!bash ./install_mecab-ko_on_colab190912.sh

In [ ]:
# Mecab 라이브러리 테스트
from konlpy.tag import Mecab
mecab = Mecab()

In [ ]:
mecab.pos("헬로키티아일랜드 인생샷")

In [ ]:
cd /content/

In [ ]:
cd mecab-ko-dic-2.1.1-20180720/

In [ ]:
ls user-dic

In [ ]:
# 사용자 사전 구성 확인
import pandas as pd

pd.read_csv("./user-dic/nnp.csv")

In [ ]:
from konlpy.tag import Mecab

mecab = Mecab()


In [ ]:
# 사용자 사전 구성을 위해 형태를 확인
with open("./user-dic/nnp.csv", 'r', encoding='utf-8') as f:
  file_data = f.readlines()

In [ ]:
file_data

In [ ]:
len(file_data[0].split(",")), len(file_data[1].split(","))

In [ ]:
word_list = pd.read_csv('/content/nnp.csv')['대우']


In [ ]:
# 기존의 형태소 분석 양상을 확인한 다음, 의도하지 않은 방식으로 단어가 분할된 경우 사용자 사전에 추가함으로 개선

with open("./user-dic/nnp.csv", 'r', encoding='utf-8') as f:
  file_data = f.readlines()

word_list = pd.read_csv('/content/nnp.csv')['대우']

for word in word_list:
  line = '{},,,,NNP,*,{},{},*,*,*,*,*\n'.format(word, 'T', word)
  file_data.append(line)

In [ ]:
file_data

In [ ]:
with open("./user-dic/nnp.csv", 'w', encoding='utf-8') as f:
  for line in file_data:
    f.write(line)

In [ ]:
with open("./user-dic/nnp.csv", 'r', encoding='utf-8') as f:
  file_new = f.readlines()
file_new

In [ ]:
# 작성한 사용자 사전을 반영
!bash ./tools/add-userdic.sh

In [ ]:
!make install

In [ ]:
# 사용자 사전을 반영한 Mecab 객체를 다시 생성
from konlpy.tag import Mecab

mecab = Mecab()

for word in word_list:
  print(mecab.pos(word))

In [ ]:
# 형태소 분석이 의도한대로 이뤄지는지 확인
mecab.pos("헬로키티아일랜드 인생샷")

In [ ]:
# 관광지별로 카테고리화된 단어 리스트를 Jeju_POI_withexplanation 파일로 저장(출처 : 대한민국 구석구석 https://korean.visitkorea.or.kr/search/search_list.do?keyword=%EC%A0%9C%EC%A3%BC%EB%8F%84)
# 유사도를 측정하기 위해 DataFrame 형식으로 저장

poi = pd.read_excel("/content/Jeju_POI_withexplanation (3).xlsx")
poi = poi[['poi','explanation']]
exp_list = poi.explanation.tolist()


In [ ]:
# 불용어 지정, 유사도 분석에 방해되는 단어들 제외
stop_words = '관광지, 대한민국, 캠페인, 제주, 제주도, 여행, 좋다, 가보다, 하다, 살다, 크다, 헬로키티아일랜드, 엉알, 서귀포김정희, 안덕, 혼인지, 랜드, 쌍용, 서귀포, 뉴파라다이스호, 상추자도, 추자도, 하추자도, 추천, 조가비, &, 100, 선, 한국, 관광, 하늬복이, 본태, 국가, 지정, 형제, 어승생, 악, 돈내코, 천지연, 정물, 월정리, 살아있다, 하멜, 상선, 표류기, 남원큰엉, 큰엉, 경승지, 제주러브랜드, 산굼부리, 성산, 용머리, 비양, 4∙3, 천제연, 정방, 돌문화, 김녕, 구역, 모구리, 속, 곳, 엉또, 이호테우, 화순, 사라, 복합, 공간, 플라넷, 각시, 학수, 곶자왈, 황우지, 대포동지삿개, 성읍, 세화, 산방, 삼양, 절물, 마라, 가파, 청보리, 수월, 모세의기적, 서건, 지미, 걸매, 금능, 소정방, 개오리, 성세기, 곽지과물, 조안, 용천, 이상한, 나라의, 엘리스, 유리의, 성, 숙박대전, 추천, 기사, 용눈이, 카멜리아힐, 다희연, 테지움, 더마파크, 다랑쉬, 도순, 성이시돌, 화조원'
stop_words = stop_words.replace(' ', '').split(',')

In [ ]:
# 리스트에서 해시태그(#) 문자를 제거
poi['explanation'] = poi['explanation'].astype(str).apply(lambda x : ' '.join(i for i in x.split('#')))
poi_df = poi
result = []

# Mecab을 활용해 추가한 사용자 사전에 따라 형태소 분석을 진행하여 반영
# 사용자 사전에 추가했으나 제대로 묶이지 않은 것들을 다시 묶어줌
# 한 음절로된 형태소를 제거하되, 형태소 중 의미를 가지는 단어들은 제외하지 않는다.

for i in range(0,len(poi['explanation'])) :
  poi['explanation'][i] = mecab.morphs(poi['explanation'][i])
  for j, word in enumerate(poi['explanation'][i]) :
    if (word == '나' and poi['explanation'][i][j+1] == '들' and poi['explanation'][i][j+2] == '이') :
      poi['explanation'][i][j] = '나들이'
      poi['explanation'][i].pop(j+1)
      poi['explanation'][i].pop(j+2)
    if (word == '피' and poi['explanation'][i][j+1] == '서') :
      poi['explanation'][i][j] = '피서'
      poi['explanation'][i].pop(j+1)
    if (word == '해' and poi['explanation'][i][j+1] == '상') :
      poi['explanation'][i][j] = '해상'
      poi['explanation'][i].pop(j+1)
    if (word == '걷' and poi['explanation'][i][j+1] == '기') :
      poi['explanation'][i][j] = '걷기'
      poi['explanation'][i].pop(j+1)
    if (word == '미' and poi['explanation'][i][j+1] == '로') :
      poi['explanation'][i][j] = '미로'
      poi['explanation'][i].pop(j+1)
    if (word == '도' and poi['explanation'][i][j+1] == '예') :
      poi['explanation'][i][j] = '도예'
      poi['explanation'][i].pop(j+1)
  

  for j, word in enumerate(poi['explanation'][i]) :
    if (word not in stop_words) :
      if (len(word) != 1 or word == '굴' or word =='꽃' or word =='봉' or word == '숲') :
        result.append(word)
      else :
        print(word) #제외되는 한 글자짜리 단어가 어떤 것인지 알기 위해 프린트
  poi['explanation'][i] = str(result).strip('[').strip(']')
  result = []


In [ ]:
# TF-IDF Vectorizer 라이브러리 임포트 한후 객체 생성
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=1, ngram_range=(1,1))

In [ ]:
# 준비된 데이터로 Fit & Transform
features = vectorizer.fit_transform(poi['explanation'])

In [ ]:
feature_names = vectorizer.get_feature_names()

In [ ]:
# DTM 생성
import numpy as np
dtm_np = np.array(features.todense())

In [ ]:
pd.DataFrame(data = dtm_np, columns = feature_names)

In [ ]:
# 코사인 유사도 구하기
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(dtm_np, dtm_np)

In [ ]:
# 관광지명 기준 인덱스 설정
indices = pd.Series(poi.index, index=poi['poi'])
indices

In [ ]:
# 확인
set(poi['explanation'])

In [ ]:
# 유사한 관광지 10개를 추천해주기 위한 메소드 작성

def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 POI의 인덱스를 가지고 옵니다.
    idx = indices[title]

    # 검색한 POI와 모든 POI의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도가 높은 순으로 POI를 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 POI를 가지고 옵니다.
    ten_selected = sim_scores[0:10]

    for i in range(0, 10):
      if (idx == ten_selected[i][0]) :
        if (i == 0) :
          ten_selected = sim_scores[1:11]
        else :
          ten_selected = sim_scores[0:i] + sim_scores[i+1:11]
    

    # 가장 유사한 10개의 POI의 인덱스를 받아옵니다.
    poi_indices = [i[0] for i in ten_selected]

    # 가장 유사한 10개의 POI의 관광지명을 반환합니다.
    return poi['poi'].iloc[poi_indices]

In [ ]:
# 가시오름 기준으로 추천 관광지 10곳 확인
get_recommendations('한라산 백록담')
